In [ ]:
!pip install -qqq simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 56.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.1/281.1 kB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 k

In [ ]:
import pandas as pd
from simpletransformers.question_answering import QuestionAnsweringModel
from sklearn.model_selection import train_test_split
import wandb

In [ ]:
data = pd.read_csv("train.csv")
train_df, eval_df = train_test_split(data, test_size=0.25, random_state=42)

In [ ]:
model_configs = {
    "bert": {
        "model_type": "bert",
        "model_name": "bert-base-uncased",
        "train_batch_size": 16,
        "eval_batch_size": 16,
        "num_train_epochs": 3,
        "learning_rate": 5e-5,
        "wandb_project": "MedQuad_QA_BERT"
    },
    "mobilebert": {
        "model_type": "bert",
        "model_name": "google/mobilebert-uncased",
        "train_batch_size": 16,
        "eval_batch_size": 16,
        "num_train_epochs": 3,
        "learning_rate": 5e-5,
        "wandb_project": "MedQuad_QA_MobileBERT"
    },
    "roberta": {
        "model_type": "roberta",
        "model_name": "roberta-base",
        "train_batch_size": 16,
        "eval_batch_size": 16,
        "num_train_epochs": 3,
        "learning_rate": 5e-5,
        "wandb_project": "MedQuad_QA_RoBERTa"
    }
}

In [ ]:
hyperparameters = [
    {"num_train_epochs": 3, "learning_rate": 5e-5, "dropout": 0.3},
    {"num_train_epochs": 3, "learning_rate": 5e-5, "dropout": 0.7}
]

In [ ]:
def preprocess_data(df):
    examples = []
    for idx, row in df.iterrows():
        context = row["Answer"]
        qas = [{"question": row["Question"], "id": idx, "answers": [{"text": row["Answer"], "answer_start": 0}]}]
        examples.append({"context": context, "qas": qas})
    return examples
for model_name, config in model_configs.items():
    for hp in hyperparameters:
        config.update(hp)
        train_examples = preprocess_data(train_df)
        eval_examples = preprocess_data(eval_df)
        wandb.init(project=config["wandb_project"], config=config)
        model = QuestionAnsweringModel(model_type=config["model_type"],
                                       model_name=config["model_name"],
                                       args=config)
        output_dir = f"{model_name} {hp}"
        model.train_model(train_examples, output_dir=output_dir)
        result = model.eval_model(eval_examples)
        wandb.log({"eval_results": result})

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

convert squad examples to features:   0%|          | 0/12305 [00:00<?, ?it/s]